In [11]:
import os
import re
import spacy
import pandas as pd
from collections import Counter

In [12]:
# Load Vietnamese NLP model
nlp = spacy.blank("vi")
nlp.add_pipe("sentencizer")

In [13]:
# Đọc file
file_path1 = "crawled/data_tuyen_sinh_phenikaa.txt"
file_path2 = "crawled/data_tuyen_sinh_ton_duc_thang.txt"

with open(file_path1, "r", encoding="utf-8") as file1:
    data1 = file1.read().strip().split("\n")

with open(file_path2, "r", encoding="utf-8") as file2:
    data2 = file2.read().strip().split("\n")

# Tạo DataFrame
df_phenikaa = pd.DataFrame(data1, columns=["phenikaa"])
df_tonducthang = pd.DataFrame(data2, columns=["tonducthang"])

In [14]:
# Làm sạch dữ liệu
def clean_text(text):
    """Xóa dấu câu (trừ . ! ?), chuyển về chữ thường"""
    # text = text.lower().strip()
    text = re.sub(r"[^\w\s.!?]", "", text)  # Giữ lại dấu chấm, chấm than, chấm hỏi
    return text

df_phenikaa["phenikaa"] = df_phenikaa["phenikaa"].apply(clean_text)
df_tonducthang["tonducthang"] = df_tonducthang["tonducthang"].apply(clean_text)

# Kiểm tra kết quả sau khi làm sạch
print(df_phenikaa.head())
print(df_tonducthang.head())

                                            phenikaa
0                             Đại học Tin tuyển sinh
1  Trường Đại học Phenikaa công bố 5 phương thức ...
2  Trường Đại học Phenikaa công bố thông tin tuyể...
3                                   1. Mã trường PKA
4                        2. Chỉ tiêu tuyển sinh 4942
                                         tonducthang
0                             Đại học Tin tuyển sinh
1  Trường ĐH Tôn Đức Thắng chỉ dành 25 chỉ tiêu x...
2  Theo phương án tuyển sinh dự kiến năm 2022 Trư...
3  Hôm nay 19.1 Trường ĐH Tôn Đức Thắng công bố p...
4  Trường ĐH Tôn Đức Thắng dự kiến tuyển 6.500 ch...


In [ ]:
def split_sentences(text):
    """Chia văn bản thành câu dựa trên dấu câu bằng spaCy"""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def count_words(text):
    """Đếm số từ bằng spaCy"""
    doc = nlp(text)
    return len([token for token in doc if token.is_alpha])

def count_stats(texts):
    """Đếm tổng số từ, câu và đoạn văn """
    total_words = sum(count_words(text) for text in texts)
    total_sentences = sum(len(split_sentences(text)) for text in texts)
    total_paragraphs = sum(len([p for p in text.split("\n") if p.strip()]) for text in texts)

    return {
        "Total Words": total_words,
        "Total Sentences": total_sentences,
        "Total Paragraphs": total_paragraphs
    }

# Thống kê cho từng tài liệu
stats_phenikaa = count_stats(df_phenikaa["phenikaa"])
stats_tonducthang = count_stats(df_tonducthang["tonducthang"])

print("PHENIKAA:", stats_phenikaa)
print("TON DUC THANG:", stats_tonducthang)

PHENIKAA: {'Total Words': 770, 'Total Sentences': 111, 'Total Paragraphs': 69}
TON DUC THANG: {'Total Words': 246, 'Total Sentences': 20, 'Total Paragraphs': 18}


In [ ]:
# Load danh sách stopwords
STOPWORDS_FILE = "vietnamese-stopwords.txt"
VIETNAMESE_STOPWORDS = set()

if os.path.exists(STOPWORDS_FILE):
    with open(STOPWORDS_FILE, "r", encoding="utf-8") as file:
        VIETNAMESE_STOPWORDS = set(file.read().strip().split("\n"))

In [ ]:
def compute_word_frequency(texts):
    """Tạo danh sách từ vựng ban đầu và tính tần suất"""
    vocab = Counter()
    for text in texts:
        words = text.split()
        vocab.update(words)
    return vocab

# Tính từ vựng ban đầu (V1)
vocab_phenikaa = compute_word_frequency(df_phenikaa["phenikaa"])
vocab_tonducthang = compute_word_frequency(df_tonducthang["tonducthang"])

# Hiển thị 10 từ phổ biến nhất
print("PHENIKAA Vocab:", vocab_phenikaa.most_common(10))
print("TON DUC THANG Vocab:", vocab_tonducthang.most_common(10))

PHENIKAA Vocab: [('tuyển', 51), ('xét', 50), ('trường', 21), ('phenikaa', 14), ('ngànhchương', 12), ('trình', 12), ('thi', 12), ('thẳng', 12), ('môn', 12), ('trở', 12)]
TON DUC THANG Vocab: [('trường', 18), ('xét', 18), ('tuyển', 16), ('thpt', 13), ('đh', 12), ('tôn', 7), ('đức', 7), ('thắng', 7), ('thi', 7), ('ngành', 7)]


In [ ]:
def remove_stopwords(vocab):
    """Loại bỏ stopwords khỏi từ vựng"""
    return {word: freq for word, freq in vocab.items() if word not in VIETNAMESE_STOPWORDS}

# Tạo từ vựng mới sau khi loại bỏ stopwords
vocab_v2_phenikaa = remove_stopwords(vocab_phenikaa)
vocab_v2_tonducthang = remove_stopwords(vocab_tonducthang)

print("PHENIKAA V2 (10 từ phổ biến nhất):", sorted(vocab_v2_phenikaa.items(), key=lambda x: x[1], reverse=True)[:10])
print("TON DUC THANG V2 (10 từ phổ biến nhất):", sorted(vocab_v2_tonducthang.items(), key=lambda x: x[1], reverse=True)[:10])


PHENIKAA V2 (10 từ phổ biến nhất): [('tuyển', 62), ('học', 60), ('xét', 47), ('sinh', 40), ('Trường', 20), ('hợp', 20), ('thí', 17), ('Đại', 15), ('môn', 15), ('trở', 15)]
TON DUC THANG V2 (10 từ phổ biến nhất): [('tuyển', 20), ('học', 19), ('xét', 18), ('sinh', 12), ('ĐH', 12), ('trình', 12), ('THPT', 11), ('chương', 11), ('kết', 11), ('Trường', 10)]


In [18]:
def extract_ngrams(texts):
    """Trích xuất unigrams, bigrams và trigrams"""
    unigrams, bigrams, trigrams = [], [], []
    
    for text in texts:
        words = text.split()
        unigrams.extend(words)
        bigrams.extend(zip(words[:-1], words[1:]))
        trigrams.extend(zip(words[:-2], words[1:-1], words[2:]))

    return unigrams, bigrams, trigrams

# Trích xuất n-grams
unigrams_p, bigrams_p, trigrams_p = extract_ngrams(df_phenikaa["phenikaa"])
unigrams_t, bigrams_t, trigrams_t = extract_ngrams(df_tonducthang["tonducthang"])

# In tất cả các n-grams
print("PHENIKAA Unigrams:", unigrams_p)
print("\nPHENIKAA Bigrams:", bigrams_p)
print("\nPHENIKAA Trigrams:", trigrams_p)

print("\nTON DUC THANG Unigrams:", unigrams_t)
print("\nTON DUC THANG Bigrams:", bigrams_t)
print("\nTON DUC THANG Trigrams:", trigrams_t)


PHENIKAA Unigrams: ['Đại', 'học', 'Tin', 'tuyển', 'sinh', 'Trường', 'Đại', 'học', 'Phenikaa', 'công', 'bố', '5', 'phương', 'thức', 'tuyển', 'sinh', 'đại', 'học', 'năm', '2022', 'Trường', 'Đại', 'học', 'Phenikaa', 'công', 'bố', 'thông', 'tin', 'tuyển', 'sinh', 'năm', '2022', 'theo', 'đó', 'Trường', 'tuyển', 'sinh', '4942', 'chỉ', 'tiêu', 'cho', '36', 'ngànhchương', 'trình', 'đào', 'tạo', 'với', '05', 'phương', 'thức', 'xét', 'tuyển.', '1.', 'Mã', 'trường', 'PKA', '2.', 'Chỉ', 'tiêu', 'tuyển', 'sinh', '4942', '3.', 'Phương', 'thức', 'tuyển', 'sinh', 'Trong', 'quá', 'trình', 'triển', 'khai', 'Nhà', 'trường', 'sẽ', 'xem', 'xét', 'và', 'điều', 'chỉnh', 'linh', 'động', 'tỷ', 'lệ', 'chỉ', 'tiêu', 'giữa', 'các', 'phương', 'thức', 'cho', 'phù', 'hợp', 'với', 'tình', 'hình', 'thực', 'tế.', '4.', 'Điều', 'kiện', 'xét', 'tuyển', 'Quy', 'định', 'xét', 'tuyển', 'không', 'áp', 'dụng', 'đối', 'với', 'phương', 'thức', 'xét', 'tuyển', 'theo', 'kết', 'quả', 'thi', 'tốt', 'nghiệp', 'THPT', 'năm', '2022', 

In [19]:
def extract_anchor_texts(texts):
    """Trích xuất anchor text từ văn bản chứa [text](link)"""
    anchor_texts = []
    pattern = r"\[(.*?)\]\((.*?)\)"
    
    for text in texts:
        matches = re.findall(pattern, text)
        anchor_texts.extend([match[0] for match in matches])
    
    return anchor_texts

# Trích xuất anchor text
anchors_p = extract_anchor_texts(df_phenikaa["phenikaa"])
anchors_t = extract_anchor_texts(df_tonducthang["tonducthang"])

print("PHENIKAA Anchor Texts:", anchors_p)
print("TON DUC THANG Anchor Texts:", anchors_t)


PHENIKAA Anchor Texts: []
TON DUC THANG Anchor Texts: []


In [20]:
def extract_named_entities(texts):
    """Thực hiện Named Entity Recognition (NER)"""
    entities = []
    for text in texts:
        doc = nlp(text)
        for ent in doc.ents:
            entities.append((ent.text, ent.label_))
    return entities

# Thực hiện NER
ner_phenikaa = extract_named_entities(df_phenikaa["phenikaa"])
ner_tonducthang = extract_named_entities(df_tonducthang["tonducthang"])

print("PHENIKAA Named Entities:", ner_phenikaa)
print("TON DUC THANG Named Entities:", ner_tonducthang)


PHENIKAA Named Entities: []
TON DUC THANG Named Entities: []
